<a href="https://colab.research.google.com/github/khairul3/BISINDO-Sign-Language-Recognition/blob/camera_test/Mediapipe_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usage example of MediaPipe Hands Solution API in Python (see also http://solutions.mediapipe.dev/hands).

In [2]:
!pip install mediapipe

     |████████████████████████████████| 36.1MB 121kB/s 


Upload any image that contains hand(s) to the Colab. We took two examples from the web: https://unsplash.com/photos/QyCH5jwrD_A and https://unsplash.com/photos/tSePVHkxUCk


##Kaggle Dataset

In [30]:
import tensorflow as tf
tf.version.VERSION

'2.5.0'

In [3]:
!pip install -q kaggle

In [4]:
#import kaggle API key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"daffashafwan","key":"9f31e017db0b036e031727d13e746047"}'}

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download -d idhamozi/indonesian-sign-language-bisindo

 99% 879M/890M [00:10<00:00, 98.5MB/s]
100% 890M/890M [00:11<00:00, 84.7MB/s]


In [9]:
!unzip indonesian-sign-language-bisindo.zip

Archive:  indonesian-sign-language-bisindo.zip
  inflating: Dataset BISINDO/datatest/A/IMG_1151.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1152.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1153.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1154.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1155.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1156.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1157.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1158.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1159.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1160.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1161.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1162.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1163.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1164.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1165.JPG  
  inflating: Dataset BISINDO/datatest/A/IMG_1166.JPG  
  inflating: Dataset BISINDO/datatest/B/IMG_1213.JPG  
  inflating: Datas

In [10]:
train_dir = 'Dataset BISINDO/datatrain/K/'

In [11]:
import cv2, os
from google.colab.patches import cv2_imshow

# Read images with OpenCV.
images = {name: cv2.imread(os.path.join(train_dir,name)) for name in os.listdir(train_dir)}


All MediaPipe Solutions Python API examples are under mp.solutions.

For the MediaPipe Hands solution, we can access this module as `mp_hands = mp.solutions.hands`.

You may change the parameters, such as `static_image_mode`, `max_num_hands`, and `min_detection_confidence`, during the initialization. Run `help(mp_hands.Hands)` to get more informations about the parameters.

In [12]:
import mediapipe as mp
mp_hands = mp.solutions.hands

help(mp_hands.Hands)

Help on class Hands in module mediapipe.python.solutions.hands:

class Hands(mediapipe.python.solution_base.SolutionBase)
 |  Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
 |  
 |  MediaPipe Hands.
 |  
 |  MediaPipe Hands processes an RGB image and returns the hand landmarks and
 |  handedness (left v.s. right hand) of each detected hand.
 |  
 |  Note that it determines handedness assuming the input image is mirrored,
 |  i.e., taken with a front-facing/selfie camera (
 |  https://en.wikipedia.org/wiki/Front-facing_camera) with images flipped
 |  horizontally. If that is not the case, use, for instance, cv2.flip(image, 1)
 |  to flip the image first for a correct handedness output.
 |  
 |  Please refer to https://solutions.mediapipe.dev/hands#python-solution-api for
 |  usage examples.
 |  
 |  Method resolution order:
 |      Hands
 |      mediapipe.python.solution_base.SolutionBase
 |      builtins.object
 |  
 |  Method

In [13]:
def mediapipe_compute(images):
  mp_drawing = mp.solutions.drawing_utils 

  results_images = []
  dict_results = dict()
  with mp_hands.Hands(
      static_image_mode=True,
      max_num_hands=2,
      min_detection_confidence=0.7) as hands:
    for name, image in images.items():
      # Convert the BGR image to RGB, flip the image around y-axis for correct 
      # handedness output and process it with MediaPipe Hands.
      results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
      image_hight, image_width, _ = image.shape
      # Print handedness (left v.s. right hand).
      if results.multi_hand_landmarks is not None and results.multi_handedness is not None and len(results.multi_handedness) == 2:
        print(f'Handedness of {name}:')
        print(results.multi_handedness)
        print(type(results.multi_handedness))
        # Draw hand landmarks of each hand.
        print(f'Hand landmarks of {name}:')

        annotated_image = cv2.flip(image.copy(), 1)
        for hand_landmarks in results.multi_hand_landmarks:
          # Print index finger tip coordinates.
          dict_results[name] = hand_landmarks.landmark
          print(
              f'Index finger tip coordinate: (',
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_hight}), '
              f'{len(hand_landmarks.landmark)})'
          )
      #   mp_drawing.draw_landmarks(
      #       annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
      # cv2_imshow(cv2.flip(annotated_image, 1))
  return dict_results

In [14]:
train_dir = 'Dataset BISINDO/datatrain/K/'

In [15]:
import cv2, os
from google.colab.patches import cv2_imshow

dicts = dict()
for i in os.listdir('Dataset BISINDO/datatrain/'):
  train_dir = 'Dataset BISINDO/datatrain/' + i
  images = {name: cv2.imread(os.path.join(train_dir,name)) for name in os.listdir(train_dir)}
  dicts[i] = mediapipe_compute(images)

Handedness of IMG_0765.JPG:
[classification {
  index: 1
  score: 0.9812120795249939
  label: "Right"
}
, classification {
  index: 0
  score: 0.9997193217277527
  label: "Left"
}
]
<class 'list'>
Hand landmarks of IMG_0765.JPG:
Index finger tip coordinate: ( 1336.3844375610352, 802.4071254730225), 21)
Index finger tip coordinate: ( 1285.7426748275757, 860.1984386444092), 21)
Handedness of IMG_1935.JPG:
[classification {
  index: 0
  score: 0.9999999403953552
  label: "Left"
}
, classification {
  index: 1
  score: 0.5278162956237793
  label: "Right"
}
]
<class 'list'>
Hand landmarks of IMG_1935.JPG:
Index finger tip coordinate: ( 858.7914505004883, 668.7910423278809), 21)
Index finger tip coordinate: ( 1351.9705352783203, 385.76840686798096), 21)
Handedness of IMG_1938.JPG:
[classification {
  index: 1
  score: 0.9986399412155151
  label: "Right"
}
, classification {
  index: 0
  score: 0.8982161283493042
  label: "Left"
}
]
<class 'list'>
Hand landmarks of IMG_1938.JPG:
Index finger 

KeyboardInterrupt: ignored

In [ ]:
dicts.items()

dict_items([('G', {}), ('P', {'IMG_1788.JPG': [x: 0.6458533406257629
y: 0.6940340399742126
z: -2.781110924843233e-05
, x: 0.5880351066589355
y: 0.6987348794937134
z: -0.10393266379833221
, x: 0.5445083975791931
y: 0.6471930146217346
z: -0.15459145605564117
, x: 0.5113351345062256
y: 0.5939719676971436
z: -0.17001605033874512
, x: 0.49195396900177
y: 0.5359310507774353
z: -0.1806664764881134
, x: 0.6385766267776489
y: 0.5438702702522278
z: -0.18823494017124176
, x: 0.6239244341850281
y: 0.4433099031448364
z: -0.22483506798744202
, x: 0.6107945442199707
y: 0.37965917587280273
z: -0.24738577008247375
, x: 0.605071485042572
y: 0.33008289337158203
z: -0.26621127128601074
, x: 0.6564453840255737
y: 0.5118102431297302
z: -0.12476766854524612
, x: 0.5802507996559143
y: 0.46758463978767395
z: -0.13684727251529694
, x: 0.5454226732254028
y: 0.5218462347984314
z: -0.1089538112282753
, x: 0.5484961271286011
y: 0.5563058257102966
z: -0.08951815217733383
, x: 0.6596359610557556
y: 0.4942421019077301

##CNN Dimulai dari sini

In [1]:
import numpy as np
import pandas as pd
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import numpy as np 
import pandas as pd 
import pandas as pd 
import numpy as np 
import scipy as sp 
import sklearn
import random 
import time 

from sklearn import preprocessing, model_selection


from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

In [2]:
from google.colab import files
csv = files.upload()

Saving test.csv to test.csv


In [7]:
import pandas as pd
data = pd.read_csv('train.csv')

In [8]:
data.loc[:, 'RIGHT_HAND_PROB':]

,RIGHT_HAND_PROB,LEFT_HAND_PROB,RIGHT_X_WRIST,RIGHT_Y_WRIST,RIGHT_Z_WRIST,RIGHT_X_THUMB_CMC,RIGHT_Y_THUMB_CMC,RIGHT_Z_THUMB_CMC,RIGHT_X_THUMB_MCP,RIGHT_Y_THUMB_MCP,RIGHT_Z_THUMB_MCP,RIGHT_X_THUMB_IP,RIGHT_Y_THUMB_IP,RIGHT_Z_THUMB_IP,RIGHT_X_THUMB_TIP,RIGHT_Y_THUMB_TIP,RIGHT_Z_THUMB_TIP,RIGHT_X_INDEX_FINGER_MCP,RIGHT_Y_INDEX_FINGER_MCP,RIGHT_Z_INDEX_FINGER_MCP,RIGHT_X_INDEX_FINGER_PIP,RIGHT_Y_INDEX_FINGER_PIP,RIGHT_Z_INDEX_FINGER_PIP,RIGHT_X_INDEX_FINGER_DIP,RIGHT_Y_INDEX_FINGER_DIP,RIGHT_Z_INDEX_FINGER_DIP,RIGHT_X_INDEX_FINGER_TIP,RIGHT_Y_INDEX_FINGER_TIP,RIGHT_Z_INDEX_FINGER_TIP,RIGHT_X_MIDDLE_FINGER_MCP,RIGHT_Y_MIDDLE_FINGER_MCP,RIGHT_Z_MIDDLE_FINGER_MCP,RIGHT_X_MIDDLE_FINGER_PIP,RIGHT_Y_MIDDLE_FINGER_PIP,RIGHT_Z_MIDDLE_FINGER_PIP,RIGHT_X_MIDDLE_FINGER_DIP,RIGHT_Y_MIDDLE_FINGER_DIP,RIGHT_Z_MIDDLE_FINGER_DIP,RIGHT_X_MIDDLE_FINGER_TIP,RIGHT_Y_MIDDLE_FINGER_TIP,...,LEFT_Z_INDEX_FINGER_DIP,LEFT_X_INDEX_FINGER_TIP,LEFT_Y_INDEX_FINGER_TIP,LEFT_Z_INDEX_FINGER_TIP,LEFT_X_MIDDLE_FINGER_MCP,LEFT_Y_MIDDLE_FINGER_MCP,LEFT_Z_MIDDLE_FINGER_MCP,LEFT_X_MIDDLE_FINGER_PIP,LEFT_Y_MIDDLE_FINGER_PIP,LEFT_Z_MIDDLE_FINGER_PIP,LEFT_X_MIDDLE_FINGER_DIP,LEFT_Y_MIDDLE_FINGER_DIP,LEFT_Z_MIDDLE_FINGER_DIP,LEFT_X_MIDDLE_FINGER_TIP,LEFT_Y_MIDDLE_FINGER_TIP,LEFT_Z_MIDDLE_FINGER_TIP,LEFT_X_RING_FINGER_MCP,LEFT_Y_RING_FINGER_MCP,LEFT_Z_RING_FINGER_MCP,LEFT_X_RING_FINGER_PIP,LEFT_Y_RING_FINGER_PIP,LEFT_Z_RING_FINGER_PIP,LEFT_X_RING_FINGER_DIP,LEFT_Y_RING_FINGER_DIP,LEFT_Z_RING_FINGER_DIP,LEFT_X_RING_FINGER_TIP,LEFT_Y_RING_FINGER_TIP,LEFT_Z_RING_FINGER_TIP,LEFT_X_PINKY_MCP,LEFT_Y_PINKY_MCP,LEFT_Z_PINKY_MCP,LEFT_X_PINKY_PIP,LEFT_Y_PINKY_PIP,LEFT_Z_PINKY_PIP,LEFT_X_PINKY_DIP,LEFT_Y_PINKY_DIP,LEFT_Z_PINKY_DIP,LEFT_X_PINKY_TIP,LEFT_Y_PINKY_TIP,LEFT_Z_PINKY_TIP
0,0.696159,0.789868,0.317047,0.682101,-0.000082,0.374849,0.733084,-0.054660,0.426938,0.716301,-0.094148,0.470238,0.686978,-0.109969,0.510129,0.671878,-0.126063,0.409878,0.605343,-0.138256,0.477187,0.505185,-0.139449,0.510828,0.444598,-0.139877,0.531550,0.388632,-0.146593,0.379099,0.561171,-0.103145,0.451296,0.519530,-0.085099,0.470536,0.528636,-0.060646,0.474887,0.531773,...,-0.223627,0.555248,0.392717,-0.235143,0.730472,0.540583,-0.135776,0.654693,0.496661,-0.114261,0.650183,0.535139,-0.065376,0.661299,0.553538,-0.041565,0.749285,0.508248,-0.090998,0.675660,0.478463,-0.064227,0.670000,0.517323,-0.029416,0.681132,0.538948,-0.015374,0.763204,0.491110,-0.050037,0.701794,0.476587,-0.026828,0.693767,0.507716,0.006464,0.702431,0.523138,0.025249
1,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.036895,0.597157,0.722845,-0.040948,0.628138,0.691566,-0.023688,0.622007,0.723869,-0.035668,0.621842,0.728538,-0.037039,0.623683,0.723645,-0.036988,0.644336,0.690390,-0.028534,0.641447,0.726246,-0.035236,0.639740,0.728482,-0.031396,0.639531,0.720309,-0.028412,0.659355,0.691445,-0.034820,0.659899,0.722455,-0.035739,0.659004,0.725035,-0.033337,0.656428,0.717193,-0.031056
2,0.000000,0.996089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.060666,0.570021,0.759889,0.066858,0.589686,0.736250,0.031347,0.582049,0.762954,0.048359,0.585021,0.767838,0.054157,0.588908,0.766861,0.059212,0.602503,0.732129,0.025553,0.594991,0.762995,0.039962,0.595775,0.765075,0.041055,0.597882,0.761425,0.041447,0.614704,0.729034,0.019032,0.610517,0.756137,0.031570,0.612060,0.758403,0.031210,0.613763,0.754853,0.033423
3,0.000000,0.999971,0.000000,0.

In [9]:
data = shuffle(data)

In [10]:
data

,IMAGE_NAME,LABEL,RIGHT_HAND_PROB,LEFT_HAND_PROB,RIGHT_X_WRIST,RIGHT_Y_WRIST,RIGHT_Z_WRIST,RIGHT_X_THUMB_CMC,RIGHT_Y_THUMB_CMC,RIGHT_Z_THUMB_CMC,RIGHT_X_THUMB_MCP,RIGHT_Y_THUMB_MCP,RIGHT_Z_THUMB_MCP,RIGHT_X_THUMB_IP,RIGHT_Y_THUMB_IP,RIGHT_Z_THUMB_IP,RIGHT_X_THUMB_TIP,RIGHT_Y_THUMB_TIP,RIGHT_Z_THUMB_TIP,RIGHT_X_INDEX_FINGER_MCP,RIGHT_Y_INDEX_FINGER_MCP,RIGHT_Z_INDEX_FINGER_MCP,RIGHT_X_INDEX_FINGER_PIP,RIGHT_Y_INDEX_FINGER_PIP,RIGHT_Z_INDEX_FINGER_PIP,RIGHT_X_INDEX_FINGER_DIP,RIGHT_Y_INDEX_FINGER_DIP,RIGHT_Z_INDEX_FINGER_DIP,RIGHT_X_INDEX_FINGER_TIP,RIGHT_Y_INDEX_FINGER_TIP,RIGHT_Z_INDEX_FINGER_TIP,RIGHT_X_MIDDLE_FINGER_MCP,RIGHT_Y_MIDDLE_FINGER_MCP,RIGHT_Z_MIDDLE_FINGER_MCP,RIGHT_X_MIDDLE_FINGER_PIP,RIGHT_Y_MIDDLE_FINGER_PIP,RIGHT_Z_MIDDLE_FINGER_PIP,RIGHT_X_MIDDLE_FINGER_DIP,RIGHT_Y_MIDDLE_FINGER_DIP,RIGHT_Z_MIDDLE_FINGER_DIP,...,LEFT_Z_INDEX_FINGER_DIP,LEFT_X_INDEX_FINGER_TIP,LEFT_Y_INDEX_FINGER_TIP,LEFT_Z_INDEX_FINGER_TIP,LEFT_X_MIDDLE_FINGER_MCP,LEFT_Y_MIDDLE_FINGER_MCP,LEFT_Z_MIDDLE_FINGER_MCP,LEFT_X_MIDDLE_FINGER_PIP,LEFT_Y_MIDDLE_FINGER_PIP,LEFT_Z_MIDDLE_FINGER_PIP,LEFT_X_MIDDLE_FINGER_DIP,LEFT_Y_MIDDLE_FINGER_DIP,LEFT_Z_MIDDLE_FINGER_DIP,LEFT_X_MIDDLE_FINGER_TIP,LEFT_Y_MIDDLE_FINGER_TIP,LEFT_Z_MIDDLE_FINGER_TIP,LEFT_X_RING_FINGER_MCP,LEFT_Y_RING_FINGER_MCP,LEFT_Z_RING_FINGER_MCP,LEFT_X_RING_FINGER_PIP,LEFT_Y_RING_FINGER_PIP,LEFT_Z_RING_FINGER_PIP,LEFT_X_RING_FINGER_DIP,LEFT_Y_RING_FINGER_DIP,LEFT_Z_RING_FINGER_DIP,LEFT_X_RING_FINGER_TIP,LEFT_Y_RING_FINGER_TIP,LEFT_Z_RING_FINGER_TIP,LEFT_X_PINKY_MCP,LEFT_Y_PINKY_MCP,LEFT_Z_PINKY_MCP,LEFT_X_PINKY_PIP,LEFT_Y_PINKY_PIP,LEFT_Z_PINKY_PIP,LEFT_X_PINKY_DIP,LEFT_Y_PINKY_DIP,LEFT_Z_PINKY_DIP,LEFT_X_PINKY_TIP,LEFT_Y_PINKY_TIP,LEFT_Z_PINKY_TIP
515,IMG_9744.JPG,J,0.000000,0.999786,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.130234,0.609898,0.559892,-0.121625,0.586514,0.495212,-0.041910,0.558355,0.489160,-0.111678,0.575278,0.551841,-0.121949,0.596908,0.583595,-0.105318,0.565275,0.521055,-0.037279,0.540566,0.518426,-0.096988,0.560029,0.573088,-0.103554,0.582419,0.600623,-0.089695,0.550859,0.549386,-0.037685,0.523657,0.517637,-0.071288,0.506219,0.516427,-0.088468,0.495936,0.505662,-0.096116
473,IMG_9694.JPG,I,0.999999,0.000000,0.640875,0.693057,0.000022,0.573658,0.661487,0.013292,0.525002,0.616946,0.016746,0.494597,0.554181,0.026179,0.503267,0.507189,0.045190,0.538240,0.548787,-0.013511,0.506619,0.507678,0.019029,0.509794,0.516182,0.050440,0.526212,0.538852,0.065113,0.566846,0.526577,-0.009638,0.529235,0.487432,0.034493,0.533923,0.498306,0.062829,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
803,IMG_1749.JPG,N,0.997757,0.000000,0.477709,0.608540,-0.000149,0.406277,0.524574,-0.066335,0.383822,0.420965,-0.115360,0.373143,0.334592,-0.156983,0.358037,0.267653,-0.188694,0.487441,0.323090,-0.111884,0.549712,0.219317,-0.158307,0.585402,0.158706,-0.189189,0.615650,0.104446,-0.210548,0.528856,0.354841,-0.108935,0.591244,0.239514,-0.158200,0.636239,0.167680,-0.196662,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
404,IMG_1055.JPG,H,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [3]:
data_predict = pd.read_csv('test.csv')

In [4]:
data_predict = shuffle(data_predict)

In [6]:
data_to_predict = data_predict
predict_alphabet = data_to_predict.LABEL 
predict_alphabet = np.array(predict_alphabet)
prediction = np.array(data_to_predict.drop(['IMAGE_NAME', 'LABEL'],axis= 1))

In [11]:
import numpy as np
import codecs, json 
from google.colab import files
data_json = data_predict.drop(['IMAGE_NAME', 'LABEL'],axis= 1)
data_json = np.array(data_json[:1])
b = data_json.tolist() # nested lists with same data, indices
file_path = "test.json" ## your path variable
json.dump(b, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)
files.download('path.json')

       IMAGE_NAME LABEL  ...  LEFT_Y_PINKY_TIP  LEFT_Z_PINKY_TIP
251  IMG_0348.JPG     R  ...          0.636392         -0.075156

[1 rows x 130 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
from google.colab import files
files.download('path.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
X = data.drop(['IMAGE_NAME', 'LABEL'], axis = 1)
X = np.array(X)
Y = data['LABEL']

In [15]:
X

array([[ 0.        ,  0.9997856 ,  0.        , ...,  0.49593601,
         0.50566196, -0.09611551],
       [ 0.99999881,  0.        ,  0.64087451, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.99775702,  0.        ,  0.47770941, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.99669224,  0.        , ...,  0.73364794,
         0.79434067,  0.03387984],
       [ 0.        ,  0.99334031,  0.        , ...,  0.55215597,
         0.57885379,  0.01069312],
       [ 0.99997729,  0.        ,  0.59702808, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
Y = np_utils.to_categorical(Y)


In [17]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y,test_size = 0.1, random_state = 0)

In [19]:
input_dim = len(data.columns) - 2

model = Sequential()
model.add(Dense(26, input_dim = input_dim , activation = 'relu'))
model.add(Dense(104, activation = 'relu'))
model.add(Dense(104, activation = 'relu'))
model.add(Dense(104, activation = 'relu'))
model.add(Dense(26, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_x, train_y, epochs = 100, batch_size = 2)

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/100
667/667 [==============================] - 2s 2ms/step - loss: 3.1709 - accuracy: 0.0599
Epoch 2/100
667/667 [==============================] - 1s 2ms/step - loss: 2.7021 - accuracy: 0.1802
Epoch 3/100
667/667 [==============================] - 1s 2ms/step - loss: 2.3996 - accuracy: 0.2585
Epoch 4/100
667/667 [==============================] - 1s 2ms/step - loss: 2.0530 - accuracy: 0.3772
Epoch 5/100
667/667 [==============================] - 1s 2ms/step - loss: 1.7796 - accuracy: 0.4741
Epoch 6/100
667/667 [==============================] - 1s 2ms/step - loss: 1.5927 - accuracy: 0.5163
Epoch 7/100
667/667 [==============================] - 1s 2ms/step - loss: 1.5350 - accuracy: 0.5377
Epoch 8/100
667/667 [==============================] - 1s 2ms/step - loss: 1.3187 - accuracy: 0.5917
Epoch 9/100
667/667 [==============================] - 1s 2ms/step - loss: 1.2151 - accuracy: 0.6327
Epoch 10/100
667/667 [==============================] - 1s 2ms/step - loss: 1.1221 - accura

In [20]:
predictions = model.predict_classes(prediction)
prediction_ = np.argmax(to_categorical(predictions), axis = 1)
prediction_ = encoder.inverse_transform(prediction_)

true_answer = 0
count = 0
for i, j in zip(prediction_ , predict_alphabet):
    print( " the neural network predict {}, and the letter to retrieve is {}".format(i,j))
    if i == j:
      true_answer += 1
    count+=1

print(count)
print(true_answer)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


 the neural network predict E, and the letter to retrieve is B
 the neural network predict V, and the letter to retrieve is V
 the neural network predict H, and the letter to retrieve is B
 the neural network predict Q, and the letter to retrieve is Q
 the neural network predict R, and the letter to retrieve is R
 the neural network predict W, and the letter to retrieve is W
 the neural network predict X, and the letter to retrieve is X
 the neural network predict C, and the letter to retrieve is C
 the neural network predict J, and the letter to retrieve is J
 the neural network predict L, and the letter to retrieve is L
 the neural network predict K, and the letter to retrieve is J
 the neural network predict H, and the letter to retrieve is H
 the neural network predict N, and the letter to retrieve is N
 the neural network predict F, and the letter to retrieve is F
 the neural network predict R, and the letter to retrieve is R
 the neural network predict U, and the letter to retrie

##Save Model buat Deploy di Cloud

In [21]:
export_path="ML/MODEL"

In [22]:
model.save(export_path)

INFO:tensorflow:Assets written to: ML/MODEL/assets


In [2]:
!ls

sample_data


In [34]:
!ls ML/MODEL

assets	keras_metadata.pb  saved_model.pb  variables


In [38]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 128)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_4'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 26)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0530 13:21:30.996346 139701468878720 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/

In [25]:
!tar -czvf model.tar.gz ML/MODEL/

ML/MODEL/
ML/MODEL/saved_model.pb
ML/MODEL/variables/
ML/MODEL/variables/variables.data-00000-of-00001
ML/MODEL/variables/variables.index
ML/MODEL/keras_metadata.pb
ML/MODEL/assets/


In [26]:
!ls

ML  model.tar.gz  sample_data  test.csv  train.csv


In [27]:
files.download('model.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>